# Looking for knickpoints

In the previous lesson, we 

In [ ]:
import pandas as pd
import geopandas as gpd
import cartopy as cp
import cartopy.crs as ccrs
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

Now read in the data

In [ ]:
df = pd.read_csv("SorbasChi_MC_MChiSegmented.csv")
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# We have to tell the geopandas data what geographic system we are in by using something called an EPSG code. 
# All major geographic projection and transformation system have this code. 
gdf.crs = "EPSG:4326" 

# The head command shows you what is in the file.
print(gdf.head())

Lets plot a single channel. This is the same as the penultamite plot in lesson 02 but I want to show the data again:

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

# First lets isolate just one of these basins. There is only basin 0 and 1
gdf_b1 = gdf[(gdf['basin_key'] == 0)]

# The main stem channel is the one with the minimum source key in this basin
# If you want to play with this a bit you can change the source number to look at different channels
min_source = np.amin(gdf_b1.source_key)
gdf_b2 = gdf_b1[(gdf_b1['source_key'] == min_source)]
#gdf_b2 = gdf_b1

# Now make channel profile plots
z = gdf_b2.elevation
x_locs = gdf_b2.flow_distance
m_chi = gdf_b2.m_chi
chi = gdf_b2.chi

# Create two subplots and unpack the output array immediately
plt.clf()
f, (ax1) = plt.subplots(1, 1)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

# Make the scatter plots
ax1.scatter(x_locs, z,s = 1, label='Longitudinal profile')
ax2.scatter(x_locs, m_chi,s = 1,c="r", label='Channel steepness')

# Some code to make sure the legend renders on the same axis
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

# These are just the labels for the figure
ax1.set_xlabel("Distance from outlet ($m$)")
ax1.set_ylabel("Elevation (m)")

ax2.set_xlabel("Distance from outlet ($m$)")
ax2.set_ylabel("Channel steepness ($k_{sn}$)")

plt.tight_layout()

The knickpoints are the ones with a high channel steepness value. We can filter the data by only picking points with a channel steepness above a certain value. So lets try that.

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

# First lets isolate just one of these basins. There is only basin 0 and 1
gdf_b1 = gdf[(gdf['basin_key'] == 0)]

# The main stem channel is the one with the minimum source key in this basin
# If you want to play with this a bit you can change the source number to look at different channels
min_source = np.amin(gdf_b1.source_key)
gdf_b2 = gdf_b1[(gdf_b1['source_key'] == min_source)]

z_all = gdf_b2.elevation
x_all = gdf_b2.flow_distance


## HERE IS THE LINE FOR SELECTING DATA WITH A HIGH m_chi
gdf_b3 = gdf_b2[(gdf_b2['m_chi'] > 80)]


# Now make channel profile plots
z = gdf_b3.elevation
x_locs = gdf_b3.flow_distance
m_chi = gdf_b3.m_chi
chi = gdf_b3.chi

# Create two subplots and unpack the output array immediately
plt.clf()
f, (ax1) = plt.subplots(1, 1)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

# Make the scatter plots
ax1.scatter(x_locs, z,s = 1, label='Knickpoints',zorder = 2)
ax1.scatter(x_all, z_all,s = 1, label='Longitudinal profile',alpha= 0.5)
ax2.scatter(x_locs, m_chi,s = 1,c="r", label='Channel steepness of knickpoins')

# Some code to make sure the legend renders on the same axis
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

# These are just the labels for the figure
ax1.set_xlabel("Distance from outlet ($m$)")
ax1.set_ylabel("Elevation (m)")

ax2.set_xlabel("Distance from outlet ($m$)")
ax2.set_ylabel("Channel steepness ($k_{sn}$)")

plt.tight_layout()

Lets try to plot the knickpoints in space:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
plt.rcParams['figure.figsize'] = [10, 10]

stamen_terrain = cimgt.Stamen('terrain-background')


# Sample the data
# First lets isolate just one of these basins. There is only basin 0 and 1
gdf_b1 = gdf[(gdf['basin_key'] == 0)]

# Set the bounds to the complete basin
bounds = gdf_b1.total_bounds

# The main stem channel is the one with the minimum source key in this basin
# If you want to play with this a bit you can change the source number to look at different channels
min_source = np.amin(gdf_b1.source_key)
gdf_b2 = gdf_b1[(gdf_b1['source_key'] == min_source)]

## HERE IS THE LINE FOR SELECTING DATA WITH A HIGH m_chi
gdf_b3 = gdf_b2[(gdf_b2['m_chi'] > 80)]

fig = plt.figure()

# Create a GeoAxes in the tile's projection.
ax = fig.add_subplot(1, 1, 1, projection=stamen_terrain.crs)

# Limit the extent of the map to a small longitude/latitude range.
ax.set_extent([bounds[0]-0.2, bounds[2]+0.2, bounds[1]-0.1, bounds[3]+0.1], crs=ccrs.Geodetic())

# Add the Stamen data at zoom level 11.
ax.add_image(stamen_terrain, 11)

# Add the channel data
gdf4 = gdf_b3.to_crs(epsg=3857)    # We have to convert the data to the same 
                               #system as the ap tiles. It happens to be this one. 
                               # This epsg code is used for all map tiles (like google maps)

# Now plot the data
gdf4.plot(ax=ax, markersize=0.5,column='m_chi', 
          zorder=10,cmap="viridis",legend=True, 
          legend_kwds={'label': "Channel steepness $k_{sn}$",'orientation': "horizontal"})
